In [41]:
# This notebook will guide you to make correct format of Huggingface dataset, in proper parquet format and visualizable in Huggingface dataset hub.
# We will take the example of the dataset "Otter-AI/MMVet" and convert it to the proper format.

In [42]:
from datasets import load_dataset
data_path = "Otter-AI/MMVet"
df = load_dataset(data_path, split="test").to_pandas()
df.head()

Using the latest cached version of the module from /mnt/bn/vl-research/workspace/boli01/.cache/huggingface/modules/datasets_modules/datasets/Otter-AI--MMVet/6d200bb35cad2b1e5ad857234f8bb5c27972216cce956017a17d17eba36f98bf (last modified on Sun Jan  7 04:41:42 2024) since it couldn't be found locally at Otter-AI/MMVet, or remotely on the Hugging Face Hub.


,id,instruction,answer,images,image_source,capability
0,v1_0,What is x in the equation?,-1<AND>-5,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,web,"[ocr, math]"
1,v1_1,What is d in the last equation?,1.25<OR>=1.25<OR>5/4,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,web,"[ocr, math]"
2,v1_2,What is x in the equation?,0.75<OR>=0.75<OR>3/4,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,web,"[ocr, math]"
3,v1_3,What is the answer to the bottom left equation?,12<OR>=12,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,web,"[ocr, spat, math]"
4,v1_4,What is the answer to the second equation on t...,7<OR>=7,[{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIH...,web,"[ocr, spat, math]"


In [48]:
from datasets import Dataset, Features, Value, Image
import pandas as pd
# Define the features for the dataset
features = Features({
    'question_id': Value(dtype='string'),
    'image': Image(),
    'question': Value(dtype='string'),
    'answer': Value(dtype='string'),
    'image_source': Value(dtype='string'),
    'capability': Value(dtype='string'),
    # Add other fields as necessary
})

df_items = {
    "question_id": [],
    "image": [],
    "question": [],
    "answer": [],
    "image_source": [],
    "capability": [],
}

for idx, row in df.iterrows():
    df_items["question_id"].append(str(row["id"]))
    image = {
        "bytes": row["images"][0]["bytes"],
        "path": ""
    }
    df_items["image"].append(image)
    df_items["question"].append(str(row["instruction"]))
    df_items["answer"].append(str(row["answer"]))
    df_items["image_source"].append(str(row["image_source"]))
    df_items["capability"].append(",".join(list(row["capability"])))
    # Add other fields as necessary

df_items = pd.DataFrame(df_items)

In [49]:
df_items.head()

,question_id,image,question,answer,image_source,capability
0,v1_0,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,What is x in the equation?,-1<AND>-5,web,"ocr,math"
1,v1_1,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,What is d in the last equation?,1.25<OR>=1.25<OR>5/4,web,"ocr,math"
2,v1_2,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,What is x in the equation?,0.75<OR>=0.75<OR>3/4,web,"ocr,math"
3,v1_3,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,What is the answer to the bottom left equation?,12<OR>=12,web,"ocr,spat,math"
4,v1_4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,What is the answer to the second equation on t...,7<OR>=7,web,"ocr,spat,math"


In [50]:
dataset = Dataset.from_pandas(df_items, features=features)

In [52]:
hub_dataset_path = "lmms-lab/MMVet"
dataset.push_to_hub(repo_id=hub_dataset_path, split="test")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/218 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/home/tiger/miniconda3/envs/lmms-eval/lib/python3.9/site-packages/huggingface_hub/lfs.py:316: UserWarning: hf_transfer is enabled but does not support uploading from bytes or BinaryIO, falling back to regular upload
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/datasets/lmms-lab/MMVet/commit/6f71fdadf11f9cfbb8048e3fab4c2687f08e01e1', commit_message='Upload dataset', commit_description='', oid='6f71fdadf11f9cfbb8048e3fab4c2687f08e01e1', pr_url=None, pr_revision=None, pr_num=None)